In [117]:
import requests
from bs4 import BeautifulSoup

url_basic = 'http://www.tripadvisor.com'
tripadv = requests.get(url_basic)
s_trip = BeautifulSoup(tripadv.text, 'lxml')
sections = s_trip.find_all('div', attrs={'class':'_2R--RBNa _39kFrNls _2PEEtTWK _3_rLKjCx _3wprI9Ge _1_nbwDp3'})

links_sections = []
for section in sections:
    if section.find('a'):
        links_sections.append('http://www.tripadvisor.com'+section.find('a').get('href'))

#sections[0].find('a').get('href')
#links_sections = [section.find('a').get('href') for section in sections]
links_sections

#¿more is a button what should i do?

['http://www.tripadvisor.com/Hotels',
 'http://www.tripadvisor.com/Rentals',
 'http://www.tripadvisor.com/Attractions',
 'http://www.tripadvisor.com/Restaurants',
 'http://www.tripadvisor.com/UserReview',
 'http://www.tripadvisor.com/ForumHome']

In [18]:
sections_adv = s_trip.find('div', attrs={'class':'_27pk-lCQ'})
print(sections_adv)

None


## SCRAPING TOP DESTINATIONS

In [19]:
url_attractions = 'https://www.tripadvisor.com/Attractions'
attractions = requests.get(url_attractions)
s_attract = BeautifulSoup(attractions.text, 'lxml')
top_destinations = s_attract.find('div', attrs={'data-track-label':'popular_destinations'}).find_all('div', attrs={'class':'poi ui_shelf_item_container ui_geo_shelf_item'})
links_top_attractions = []
for destination in top_destinations:
    if destination.find('a'):
        links_top_attractions.append('http://www.tripadvisor.com'+destination.find('a').get('href'))
        
#links_top_attractions

links_top_attractions



['http://www.tripadvisor.com/Attractions-g31350-Activities-Scottsdale_Arizona.html',
 'http://www.tripadvisor.com/Attractions-g32655-Activities-Los_Angeles_California.html',
 'http://www.tripadvisor.com/Attractions-g45963-Activities-Las_Vegas_Nevada.html',
 'http://www.tripadvisor.com/Attractions-g60745-Activities-Boston_Massachusetts.html']

In [44]:
def content_info(soup, activities_list_urls):
    #for link in links_top_attractions:
    dir_results = {}
    try:
        
        #Extracting title
        title = soup.find('div', attrs = {'class':'_1l9azAvU'})
        if title is not None:
            dir_results['title'] = title.h1.text
            
        else:
            title = soup.find('div', attrs={'class':'display_center ui_container'})
            if title is not None:
                dir_results['title'] = title.h1.text
            else:
                dir_results['title'] = None
        
        #Extracting Activities
        #fixed for Boston
        activities = soup.find_all('span', attrs = {'class':'_2e_OvRJN'})
                
        #if activities is not None:
        if len(activities) != 0:
            activities_list = [activitie.h3.text for activitie in activities]
            dir_results['activities'] = activities_list
        else:  
          
            activities = soup.find_all('div', attrs = {'class':'_1lY2qyk3'})
            
            if activities is not None:
                activities_list = [activitie.h3.text for activitie in activities]
                dir_results['activities'] = activities_list
            else:
                dir_results['activities'] = None
                
        #Extracting Images
        
        images_media = soup.find_all('a', attrs={'class':'_1zqXepI-'})
        
        if len(images_media) != 0:
            images = [img.find('img').get('data-url') for img in images_media]
            dir_results['images'] = [img_fin.replace('https','http') for img_fin in images]
        else:
            images_media = soup.find_all('a', attrs={'class':'_1o7ZDa9J'})
            if images_media:
                images = [img.img.get('data-url') for img in images_media]
                dir_results['images'] = [img_fin.replace('https','http') for img_fin in images]
            
            else:
                dir_results['images'] = None
        
        #Extracting Content
        
        
        content_overview = get_content(activities_list_urls)
        if content_overview:
            dir_results['overviews'] = content_overview
        else:
            dir_results['overviews'] = None
        
            
        
        #print(dir_results)
      
        return dir_results
                                         
          
    except Exception as e:
        print('Error')
        print(e)
        print('\n')
        

 

## GET CONTENT

In [114]:


def get_links_activities(url_city):
    r_url_city = requests.get(url_city)
    if r_url_city.status_code == 200:
        s_city =  BeautifulSoup(r_url_city.text, 'lxml')
        activities_class = s_city.find_all('div', attrs={'class':'_1sXmOkVY'})
        links_activities = [(url_basic+activity.find('a').get('href')) for activity in activities_class]
                            #.replace('https','http') for activity in activities_class]
    
    return links_activities
    

def get_content(activities_list):
    activities_overview = []
    for activity in activities_list:
        r_activity = requests.get(activity)
        if r_activity.status_code == 200:
            r_activity_soup = BeautifulSoup(r_activity.text,'lxml')
            activity_content =  r_activity_soup.find('div', attrs={'class':'AvpaRatK'})
            if activity_content:
                activities_overview.append(activity_content.find('span').text)
            else:
                activities_overview.append('Activity N/A')
            
    
    return activities_overview
        
        #links_Activities =  s_content_activities.find
        

In [60]:
url_city = links_top_attractions[2]
test = get_links_activities(url_city)
print(test)
print(get_content(test).find('span').text)

['https://www.tripadvisor.com/AttractionProductReview-g45963-d12152754-The_High_Roller_at_The_LINQ_Ticket-Las_Vegas_Nevada.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d11451833-Las_Vegas_Helicopter_Night_Flight_with_Optional_VIP_Transportation-Las_Vegas_Nevada.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d11449287-Las_Vegas_Super_Saver_Madame_Tussauds_with_Gondola_Boat_Ride-Las_Vegas_Nevada.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d15358897-Fly_LINQ_Zipline_at_The_LINQ-Las_Vegas_Nevada.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d11461471-Grand_Canyon_West_Rim_with_Optional_Skywalk_Ticket_and_Hoover_Dam_Photo_Stop-Las_Ve.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d15658319-Emerald_Cave_Kayak_Tour_on_The_Colorado_River_from_Las_Vegas-Las_Vegas_Nevada.html', 'https://www.tripadvisor.com/AttractionProductReview-g45963-d21208919-Masters_of_Magic_Show_Tickets_at_Las_Vegas_Ma

AttributeError: 'NoneType' object has no attribute 'find'

In [29]:
url_city
helicopter = test[1]
r_helicopter =  requests.get(helicopter)
s_helicopter = BeautifulSoup(r_helicopter.text,'html.parser')
content = s_helicopter.find('div', attrs={'data-tab':'TABS_OVERVIEW'})
print(content)

<div data-tab="TABS_OVERVIEW"><div class="_31J0KY8Y" data-automation="overviewHeader"><span class="_16nAF3ka">Overview<!-- --> </span></div><div class="_2z6V2LzK pLYpaKkl" data-automation="overviewContent"><div class="_2zqaS_NY"><div class="_39-Vl9rc"><div class="nqu_DTdQ A_7_yDE7" data-skeleton="@ta/styleguide.skeleton" style="height:32px;width:100%"></div><div class="nqu_DTdQ A_7_yDE7" data-skeleton="@ta/styleguide.skeleton" style="height:32px;width:100%"></div><div class="nqu_DTdQ A_7_yDE7" data-skeleton="@ta/styleguide.skeleton" style="height:32px;width:100%"></div></div></div></div></div>


### TESTS MEDIA

In [ ]:
string_test = ['https://test.com','http://test2.com']
for string in string_test:
    print(string.replace('https','http'))

In [ ]:
from IPython.display import Image

url_test_boston = 'http://www.tripadvisor.com/Attractions-g60745-Activities-Boston_Massachusetts.html'
request_boston = requests.get(url_test_boston)
s_boston = BeautifulSoup(request_boston.text, 'lxml')    

media_boston = s_boston.find('a', attrs={'class':'_1o7ZDa9J'}).img
media_boston.get('data-url')

#image_req = requests.get(media_boston)

#image_req.status_code
#Image(image_req.content)
#media_content = s_content.find_all('a', attrs={'class':'_1zqXepI-'})


In [ ]:
content_test = requests.get('https://www.tripadvisor.com/Attractions-g60745-Activities-Boston_Massachusetts.html')
if content.status_code == 200:
    s_content_test = BeautifulSoup(content_test.text,'lxml')
    activities_test = s_content_test.find_all('div', attrs = {'class':'_1lY2qyk3'})
    activities_list = [activitie.h3.text for activitie in activities_test]
    print(activities_list)



### CALL CONTENT_INFO

In [115]:
from IPython.display import Image
for link in links_top_attractions: #Top attractions refers to the place: Scottsdale, LA, etc
    content = requests.get(link)
    if content.status_code == 200:
        s_content = BeautifulSoup(content.text,'lxml')
        activities_content_links = get_links_activities(link)
        #print(activities_content_links)
        
        #print(get_content(activities_content_links))
        results = content_info(s_content, activities_content_links)
        print(results)
        
        #sites_images = results['images']
        #for img in sites_images:
         #   image_req = requests.get(img)
            #print(image_req.status_code)
            #Image(image_req.content)
                        

            


        #images_list = results['images']
        #images_list
        #for image in images_list:
         #   image_req = requests.get(image)
       





{'title': 'Things to Do in Scottsdale', 'activities': ['Sonoran Desert Jeep Tour at Sunset', 'Medieval Times Dinner & Tournament Admission Ticket in Scottsdale', '2-Hour Scottsdale Segway Tours - 10am Departure', 'Half Day Sonoran Desert Mountain Biking Tour', 'Phoenix Indoor Skydiving Experience with 2 Flights & Personalized Certificate', '1-Hour Private Chemical Lab Escape Room Experience', 'Grand Canyon and Sedona Day Adventure from Scottsdale or Phoenix', '1.5 Hour Scottsdale Segway Tour', '2-Hour Arizona Desert Guided E-Bike Tour', 'Sonoran Desert 2 Hour Guided ATV Adventure', 'Sonoran Desert Jeep Tour - Morning or Afternoon', 'Rappelling Adventure in Scottsdale ', 'Party Bike Pub Crawl of Old Town Scottsdale', 'Horse Country Estate and Vineyard Wine Tasting Tour', 'Arizona Food Tours- A Taste of Old Town Scottsdale ', 'H2 Hummer VIP Sunset Wine and Cheese Tour for Couples ', '2-Hour Scottsdale Segway Tours - 4pm Departure', 'Scottsdale Segway Tour - Evening', 'Scottsdale City Tou

{'title': 'Things to Do in Las Vegas', 'activities': ['The High Roller at The LINQ Ticket', 'Las Vegas Helicopter Night Flight with Optional VIP Transportation', 'Las Vegas Super Saver: Madame Tussauds with Gondola Boat Ride', 'Fly LINQ Zipline at The LINQ', 'Grand Canyon West Rim with Optional Skywalk Ticket and Hoover Dam Photo Stop', 'Emerald Cave Kayak Tour on The Colorado River from Las Vegas', 'Masters of Magic Show Tickets at Las Vegas Magic Theater', 'Big Bus Las Vegas Hop-On Hop-Off Tour', 'Historic Downtown Walking Tour in Las Vegas', 'Christmas Magic Show Ticket at Las Vegas Magic Theater', 'Private Tour: 7-Day Highlight Tour of Southwest Mighty Five', 'Skip-The-Line Ticket to Experience Magic Up Close Show', 'Extreme RZR Tour of Hidden Valley and Primm from Las Vegas', 'Grand Canyon West Rim Bus Tour from Las Vegas with Optional Upgrades', 'Las Vegas Hot Air Balloon Ride', 'Grand Canyon South Rim Bus Tour', 'Madame Tussauds Las Vegas Admission Ticket', 'Eiffel Tower Viewing

{'title': 'Things to Do in Boston', 'activities': ['Boston Whale Watching Cruise by High-Speed Catamaran', 'Boston Old Town Hop On Hop Off Trolley Tour', 'Boston Tea Party Ships & Museum Admission', 'Boston Classic: Duck Boat Sightseeing Tour', 'Boston Ghosts & Gravestones Night-Time Trolley Tour', "Martha's Vineyard Day Trip from Boston with Optional Island Tour (Small Group)", 'Walking Tour of Freedom Trail History (Small Group)', 'New England Aquarium Admission in Boston', 'Boston Odyssey Brunch Cruise', 'Sightseeing Day Sail around Boston Harbor', 'Sunset Cruise', 'Boston Odyssey Dinner Cruise', 'Boston Odyssey Dinner Cruise', "Boston's North End Small-Group Walking Food Tour", 'Go Boston All-Inclusive Pass', "Fresh Pasta Making Lesson in Boston's North End (Homemade Pasta in Little Italy)", 'Boston to New Hampshire Fall Foliage White Mountains Tour', 'Boston Walk Through History Private Tour All-Inclusive', 'Historic Discovery Guided Boston Mini-Bus Tour', "Martha's Vineyard Day T

In [110]:
links_final_test = get_links_activities(links_top_attractions[0])

over_list = []
for link in links_final_test:
    r_activity_test = requests.get(link)
    #.replace('https','http')
    s_r_activity_test = BeautifulSoup(r_activity_test.text,'lxml')
    #print(links_final_test[0].replace('https','http'))
    activity_content_test =  s_r_activity_test.find('div', attrs={'class':'AvpaRatK'})
    if activity_content_test:
        over_list.append(activity_content_test.find('span').text)
       
    else:
        over_list.append(f'No overview for {link}')
        
over_list


['No overview for https://www.tripadvisor.com/AttractionProductReview-g31350-d19011851-Sonoran_Desert_Jeep_Tour_at_Sunset-Scottsdale_Arizona.html',
 'No overview for https://www.tripadvisor.com/AttractionProductReview-g31350-d19147851-Medieval_Times_Dinner_Tournament_Admission_Ticket_in_Scottsdale-Scottsdale_Arizona.html',
 'Begin your Scottsdale adventure with a comprehensive overview of the city’s main sights on a small-group Segway tour. The easy-to-use mode of transport enables you to cover a huge area of Scottsdale and be back in time for lunch. Stop at scenic locations and learn interesting facts about your surroundings. The excursion is a great opportunity to get insider tips on things to do during the rest of your visit.',
 "Seeing the Sonoran desert on two wheels can be memorable. We have lots of options for any level of rider. If you have not been on a bike for a while don't worry we can help you rediscover that talent. If you are a road cyclist and or an intermediate and exp

# SCRAPY

In [ ]:
try:
    import scrapy
except:
    !pip3 install scrapy
    import scrapy

In [ ]:
class Spidertrip(scrapy.Spider):
    name = 'spidertrip'
    allowed_domains = ['tripadvisor.com']
    custom_settings = {'FEED_FORMAT': 'json', 
                      'FEED_URI':'results.json'}
    #start_urls = 